In [2]:
!pip install rembg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|████████████████████████████████████████| 176M/176M [00:00<00:00, 118GB/s]


In [4]:
# Required Libraries:
# pip install Pillow numpy matplotlib rembg scipy onnxruntime
from google.colab import drive

# --- 1. Mount Google Drive ---
drive.mount('/content/drive')
from rembg import remove
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

# 1. Paths to Your Images
CLOTH_IMAGE_PATH = '/content/drive/MyDrive/00073_00.jpg'      # T-shirt/cloth image
LOGO_IMAGE_PATH = '/content/drive/MyDrive/eagle.jpg'       # Logo image (preferably on a plain background)
RESULT_IMAGE_PATH = 'tshirt_with_logo.png'                    # Output result

# 2. Remove Logo Background (AI-based)
with open(LOGO_IMAGE_PATH, 'rb') as i:
    input_data = i.read()
    result_data = remove(input_data)
    with open('logo_no_bg.png', 'wb') as o:
        o.write(result_data)
logo_img = Image.open('logo_no_bg.png').convert('RGBA')

# 3. Load Cloth Image (RGBA)
cloth_img = Image.open(CLOTH_IMAGE_PATH).convert('RGBA')
shirt_w, shirt_h = cloth_img.size

# 4. Resize and Center Logo
logo_max_width = int(shirt_w * 0.4)  # Make logo max 50% of shirt width
logo_aspect = logo_img.width / logo_img.height
logo_height = int(logo_max_width / logo_aspect)
logo_resized = logo_img.resize((logo_max_width, logo_height), Image.LANCZOS)
pos_x = (shirt_w - logo_max_width) // 2
pos_y = (shirt_h - logo_height) // 2

# 5. Feather Logo Edges for a "Printed" Look
logo_arr = np.array(logo_resized)
alpha = logo_arr[:, :, 3]
alpha_blur = gaussian_filter(alpha, sigma=2)
logo_arr[:, :, 3] = alpha_blur
logo_blurred = Image.fromarray(logo_arr)

# 6. Composite Logo onto Shirt with Smooth Alpha
output_img = cloth_img.copy()
output_img.paste(logo_blurred, (pos_x, pos_y), logo_blurred)
output_img.save(RESULT_IMAGE_PATH)

# 7. Display Original Cloth, Logo, and Result Side by Side
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# Show original cloth image
axs[0].imshow(cloth_img)
axs[0].set_title("Original Cloth Image")
axs[0].axis('off')

# Show extracted logo (background removed)
# For clearer viewing, place logo on a white background
white_bg = Image.new('RGBA', logo_resized.size, (255,255,255,255))
white_bg.paste(logo_blurred, (0,0), logo_blurred)
axs[1].imshow(white_bg)
axs[1].set_title("Logo with Background Removed")
axs[1].axis('off')

# Show result (logo on t-shirt)
axs[2].imshow(output_img)
axs[2].set_title("T-Shirt with Printed Logo")
axs[2].axis('off')

plt.tight_layout()
plt.show()